This notebook is run from within the directory holding datasets stored locally. No Gucchi back and forth to the buckets.  Using dataframes to look at column headers and generate a preferred, fully joined set of dat. 

In [1]:
import pandas as pd
import numpy as np
import s3fs
import os
import io
import boto3

In [2]:
df = pd.read_csv('decades_tcc_ceds_music.csv')

In [3]:
df.head()

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
0,0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,95,0.000598,0.063746,0.000598,...,0.380299,0.117175,0.357739,0.454119,0.997992,0.901822,0.339448,0.137110,sadness,1.0
1,4,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,51,0.035537,0.096777,0.443435,...,0.001284,0.001284,0.331745,0.647540,0.954819,0.000002,0.325021,0.263240,world/life,1.0
2,6,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,24,0.002770,0.002770,0.002770,...,0.002770,0.225422,0.456298,0.585288,0.840361,0.000000,0.351814,0.139112,music,1.0
3,10,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54,0.048249,0.001548,0.001548,...,0.225889,0.001548,0.686992,0.744404,0.083935,0.199393,0.775350,0.743736,romantic,1.0
4,12,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48,0.001350,0.001350,0.417772,...,0.068800,0.001350,0.291671,0.646489,0.975904,0.000246,0.597073,0.394375,romantic,1.0


In [4]:
df.iloc[2]['lyrics']

'sweetheart send letter goodbye secret feel better wake dream think real false emotions feel heartaches hang long blue get bluer song remember cloudy hair'

In [5]:
df.describe(include='all')

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
count,28372.000000,28372,28372,28372.000000,28372,28372,28372.000000,28372.000000,28372.000000,28372.000000,...,28372.000000,28372.000000,28372.000000,28372.000000,2.837200e+04,28372.000000,28372.000000,28372.000000,28372,28372.000000
unique,NaN,5426,23689,NaN,7,28372,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN
top,NaN,johnny cash,tonight,NaN,pop,baby design comfort spread towel bring lotion ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sadness,NaN
freq,NaN,190,17,NaN,7042,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6096,NaN
mean,42946.323558,NaN,NaN,1990.236888,NaN,NaN,73.028444,0.021112,0.118396,0.120973,...,0.129389,0.030996,0.533348,0.665249,3.392347e-01,0.080049,0.532864,0.569875,NaN,0.425187
std,24749.325492,NaN,NaN,18.487463,NaN,NaN,41.829831,0.052370,0.178684,0.172200,...,0.181143,0.071652,0.173218,0.108434,3.267143e-01,0.211245,0.250972,0.244385,NaN,0.264107
min,0.000000,NaN,NaN,1950.000000,NaN,NaN,1.000000,0.000291,0.000284,0.000291,...,0.000284,0.000289,0.005415,0.000000,2.811248e-07,0.000000,0.000000,0.000000,NaN,0.014286
25%,20391.250000,NaN,NaN,1975.000000,NaN,NaN,42.000000,0.000923,0.001120,0.001170,...,0.001144,0.000993,0.412975,0.595364,3.423598e-02,0.000000,0.329143,0.380361,NaN,0.185714
50%,45405.500000,NaN,NaN,1991.000000,NaN,NaN,63.000000,0.001462,0.002506,0.006579,...,0.005263,0.001754,0.538612,0.679050,2.259028e-01,0.000085,0.539365,0.580567,NaN,0.414286
75%,64090.500000,NaN,NaN,2007.000000,NaN,NaN,93.000000,0.004049,0.192608,0.197793,...,0.235113,0.032622,0.656666,0.749026,6.325298e-01,0.009335,0.738252,0.772766,NaN,0.642857


In [6]:
df[df.duplicated(subset=['artist_name','track_name'])]

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age


In [7]:
df.genre.unique()

array(['pop', 'country', 'blues', 'jazz', 'reggae', 'rock', 'hip hop'],
      dtype=object)

In [8]:
!ls

Untitled.ipynb             genres_lyrics_data.csv
Wrangle1.ipynb             labeled_lyrics_cleaned.csv
album_details_25k.csv      lyrics_25k.csv
boto_keys.env              short_name.csv
decades_tcc_ceds_music.csv songs_details_25k.csv
genres_artists_data.csv


In [9]:
df1=pd.read_csv('genres_artists_data.csv')

In [10]:
df1.head()

,Artist,Songs,Popularity,Link,Genre,Genres
0,10000 Maniacs,110,0.3,/10000-maniacs/,Rock,Rock; Pop; Electronica; Dance; J-Pop/J-Rock; G...
1,12 Stones,75,0.3,/12-stones/,Rock,Rock; Gospel/Religioso; Hard Rock; Grunge; Roc...
2,311,196,0.5,/311/,Rock,Rock; Surf Music; Reggae; Ska; Pop/Rock; Rock ...
3,4 Non Blondes,15,7.5,/4-non-blondes/,Rock,Rock; Pop/Rock; Rock Alternativo; Grunge; Blue...
4,A Cruz Está Vazia,13,0.0,/a-cruz-esta-vazia/,Rock,Rock


In [11]:
df1.describe(include='all')

,Artist,Songs,Popularity,Link,Genre,Genres
count,3242,3242.000000,3242.000000,3242,3242,3238
unique,2940,NaN,NaN,2940,6,1777
top,MC Novinho,NaN,NaN,/edu-gueda/,Rock,Sertanejo
freq,3,NaN,NaN,3,797,295
mean,NaN,53.907465,1.522455,NaN,NaN,NaN
std,NaN,78.212829,6.965121,NaN,NaN,NaN
min,NaN,0.000000,0.000000,NaN,NaN,NaN
25%,NaN,4.000000,0.000000,NaN,NaN,NaN
50%,NaN,17.000000,0.000000,NaN,NaN,NaN
75%,NaN,77.000000,0.500000,NaN,NaN,NaN


In [12]:
df1.Genre.unique()

array(['Rock', 'Pop', 'Hip Hop', 'Samba', 'Funk Carioca', 'Sertanejo'],
      dtype=object)

In [13]:
df1.keys()

Index(['Artist', 'Songs', 'Popularity', 'Link', 'Genre', 'Genres'], dtype='object')

In [14]:
df1.Genres.unique()

array(['Rock; Pop; Electronica; Dance; J-Pop/J-Rock; Gospel/Religioso; Infantil; Emocore',
       'Rock; Gospel/Religioso; Hard Rock; Grunge; Rock Alternativo; Hardcore; Punk Rock; Chillout; Electronica; Heavy Metal; Metal; World Music; Axé; Emocore',
       'Rock; Surf Music; Reggae; Ska; Pop/Rock; Rock Alternativo; Hardcore',
       ...,
       'Romântico; Pagode; Samba; Country; Hardcore; Tecnopop; Axé; Funk Carioca; Black Music',
       'Pagode; Romântico; Samba; Trilha Sonora; Black Music; Forró; Electronica; Funk Carioca; Sertanejo; Funk; Rock Alternativo; Jovem Guarda; Chillout; Pop/Rock; Disco; Infantil; World Music',
       'Romântico; Pagode; Samba; Sertanejo; Funk Carioca; Instrumental; Forró; Black Music; Axé; Funk; Fado; R&B; Pop/Punk; Blues; Gospel/Religioso; Electronica; Reggae; Rap; Emocore'],
      dtype=object)

In [15]:
df1.iloc[3]['Link']

'/4-non-blondes/'

In [28]:
df2 = pd.read_csv('genres_lyrics_data.csv')

In [29]:
df2

,ALink,SName,SLink,Lyric,Idiom
0,/10000-maniacs/,More Than This,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...,ENGLISH
1,/10000-maniacs/,Because The Night,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close...",ENGLISH
2,/10000-maniacs/,These Are Days,/10000-maniacs/these-are-days.html,These are. These are days you'll remember. Nev...,ENGLISH
3,/10000-maniacs/,A Campfire Song,/10000-maniacs/a-campfire-song.html,"A lie to say, ""O my mountain has coal veins an...",ENGLISH
4,/10000-maniacs/,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,Trudging slowly over wet sand. Back to the ben...,ENGLISH
...,...,...,...,...,...
209517,/zeca-pagodinho/,Vou Botar Teu Nome Na Macumba,/zeca-pagodinho/vou-botar-teu-nome-na-macumba....,Eu vou botar teu nome na macumba. Vou procurar...,PORTUGUESE
209518,/zeca-pagodinho/,Vou Procurar Esquecer,/zeca-pagodinho/vou-procurar-esquecer.html,Vou procurar um novo amor na minha vida. Porqu...,PORTUGUESE
209519,/zeca-pagodinho/,Vou Ver Juliana,/zeca-pagodinho/vou-ver-juliana.html,Quando a mare vazá. Vou vê juliana. Vou vê jul...,PORTUGUESE
209520,/zeca-pagodinho/,Yaô Cadê A Samba / Outro Recado / Hino,/zeca-pagodinho/yao-cade-a-samba-outro-recado-...,"Ô Yaô. Yaô, cadê a samba?. Está mangando na cu...",PORTUGUESE


In [30]:
df2[df2.duplicated(subset=['ALink','SName'])]

,ALink,SName,SLink,Lyric,Idiom
23,/10000-maniacs/,A Campfire Song,/10000-maniacs/a-campfire-song.html,"A lie to say, ""O my mountain has coal veins an...",ENGLISH
24,/10000-maniacs/,A Room For Everything,/10000-maniacs/a-room-for-everything.html,"You were looking away from me, western skies c...",ENGLISH
25,/10000-maniacs/,Across The Fields,/10000-maniacs/across-the-fields.html,"Well they left then in the morning, a hundred ...",ENGLISH
26,/10000-maniacs/,All That Never Happens,/10000-maniacs/all-that-never-happens.html,"She walks alone on the brick lane,. the breeze...",ENGLISH
29,/10000-maniacs/,Anthem For Doomed Youth,/10000-maniacs/anthem-for-doomed-youth.html,For whom do the bells toll. When sentenced to ...,ENGLISH
...,...,...,...,...,...
209477,/zeca-pagodinho/,Seu Balancê,/zeca-pagodinho/seu-balance.html,Quando o canto da sereia. Reluziu no seu olhar...,PORTUGUESE
209485,/zeca-pagodinho/,Spc,/zeca-pagodinho/spc.html,Precisei de roupa nova. Mas sem prova de salar...,PORTUGUESE
209506,/zeca-pagodinho/,Vacilão,/zeca-pagodinho/vacilao.html,Aquilo que era mulher. Pra não te acordar cedo...,PORTUGUESE
209512,/zeca-pagodinho/,Verdade,/zeca-pagodinho/verdade-1.html,Descobri que te amo demais. Descobri em você m...,PORTUGUESE


In [19]:
df2.describe(include='all')

,ALink,SName,SLink,Lyric,Idiom
count,209522,209522,209522,209484,206375
unique,2993,128083,167499,164789,47
top,/chris-brown/,Intro,/edu-gueda/menina.html,Instrumental,ENGLISH
freq,1176,80,6,592,114723


In [20]:
df2.Idiom.unique()

array(['ENGLISH', nan, 'PORTUGUESE', 'SPANISH', 'ITALIAN', 'FRENCH',
       'KINYARWANDA', 'DANISH', 'NORWEGIAN', 'GERMAN', 'INDONESIAN',
       'SWAHILI', 'FINNISH', 'SLOVAK', 'BASQUE', 'ESTONIAN', 'SERBIAN',
       'CROATIAN', 'BOSNIAN', 'IRISH', 'CATALAN', 'KURDISH', 'SUNDANESE',
       'HUNGARIAN', 'DUTCH', 'AFRIKAANS', 'ICELANDIC', 'MALAY', 'SESOTHO',
       'SWEDISH', 'WELSH', 'TAGALOG', 'POLISH', 'GALICIAN',
       'HAITIAN_CREOLE', 'KOREAN', 'GANDA', 'HMONG', 'NYANJA', 'RUSSIAN',
       'ARABIC', 'TURKISH', 'MALAGASY', 'JAPANESE', 'SLOVENIAN', 'CZECH',
       'CEBUANO', 'ROMANIAN'], dtype=object)

In [42]:
df2.shape

(167505, 5)

In [23]:
df2.ndim

2

In [43]:
df2.keys()

Index(['ALink', 'SName', 'SLink', 'Lyric', 'Idiom'], dtype='object')

In [46]:
df3 = df2.drop_duplicates

In [25]:
df2[df2.duplicated(subset=['ALink','SName'])]

,ALink,SName,SLink,Lyric,Idiom


In [47]:
df2.describe(include='all')

,ALink,SName,SLink,Lyric,Idiom
count,167505,167505,167505,167469,164826
unique,2993,128083,167492,164783,47
top,/elvis-presley/,Intro,/dionne-warwick/theme-from-valley-of-the-dolls...,Instrumental,ENGLISH
freq,759,71,2,548,91619


In [48]:
df2

,ALink,SName,SLink,Lyric,Idiom
0,/10000-maniacs/,More Than This,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...,ENGLISH
1,/10000-maniacs/,Because The Night,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close...",ENGLISH
2,/10000-maniacs/,These Are Days,/10000-maniacs/these-are-days.html,These are. These are days you'll remember. Nev...,ENGLISH
3,/10000-maniacs/,A Campfire Song,/10000-maniacs/a-campfire-song.html,"A lie to say, ""O my mountain has coal veins an...",ENGLISH
4,/10000-maniacs/,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,Trudging slowly over wet sand. Back to the ben...,ENGLISH
...,...,...,...,...,...
209516,/zeca-pagodinho/,Vôo de Paz,/zeca-pagodinho/voo-de-paz.html,Há qualquer coisa entre nós. Que nos priva de ...,PORTUGUESE
209518,/zeca-pagodinho/,Vou Procurar Esquecer,/zeca-pagodinho/vou-procurar-esquecer.html,Vou procurar um novo amor na minha vida. Porqu...,PORTUGUESE
209519,/zeca-pagodinho/,Vou Ver Juliana,/zeca-pagodinho/vou-ver-juliana.html,Quando a mare vazá. Vou vê juliana. Vou vê jul...,PORTUGUESE
209520,/zeca-pagodinho/,Yaô Cadê A Samba / Outro Recado / Hino,/zeca-pagodinho/yao-cade-a-samba-outro-recado-...,"Ô Yaô. Yaô, cadê a samba?. Está mangando na cu...",PORTUGUESE


But, How to delete all the not ENGLISH idioms??

In [50]:
print(df2[df2['Idiom']=='ENGLISH'])

                   ALink                                SName  \
0        /10000-maniacs/                       More Than This   
1        /10000-maniacs/                    Because The Night   
2        /10000-maniacs/                       These Are Days   
3        /10000-maniacs/                      A Campfire Song   
4        /10000-maniacs/              Everyday Is Like Sunday   
...                  ...                                  ...   
207611           /sambo/              Smells Like Teen Spirit   
207624           /sambo/                              Valerie   
207628           /sambo/                           Wake Me Up   
207792       /seu-jorge/                                Don't   
209285  /zeca-pagodinho/  Deixa A Vida Me Levar (translation)   

                                                    SLink  \
0                      /10000-maniacs/more-than-this.html   
1                   /10000-maniacs/because-the-night.html   
2                      /10000-maniac

In [51]:
english_df2 = df2['Idiom']=='ENGLISH'

In [52]:
english_df2.describe(include='all')

count     167505
unique         2
top         True
freq       91619
Name: Idiom, dtype: object

In [53]:
english_df2.head()

0    True
1    True
2    True
3    True
4    True
Name: Idiom, dtype: bool

In [54]:
englishh_df2 = (df2[df2['Idiom']=='ENGLISH'])

In [55]:
englishh_df2.head()

,ALink,SName,SLink,Lyric,Idiom
0,/10000-maniacs/,More Than This,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...,ENGLISH
1,/10000-maniacs/,Because The Night,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close...",ENGLISH
2,/10000-maniacs/,These Are Days,/10000-maniacs/these-are-days.html,These are. These are days you'll remember. Nev...,ENGLISH
3,/10000-maniacs/,A Campfire Song,/10000-maniacs/a-campfire-song.html,"A lie to say, ""O my mountain has coal veins an...",ENGLISH
4,/10000-maniacs/,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,Trudging slowly over wet sand. Back to the ben...,ENGLISH


In [56]:
englishh_df2.describe(include='all')

,ALink,SName,SLink,Lyric,Idiom
count,91619,91619,91619,91619,91619
unique,1198,70261,91608,90794,1
top,/elvis-presley/,Intro,/bee-gees/our-love-dont-throw-it-all-away.html,[This song is an instrumental.].,ENGLISH
freq,747,51,2,16,91619


In [59]:
englishh_df2.dtypes

ALink    object
SName    object
SLink    object
Lyric    object
Idiom    object
dtype: object

In [60]:
type(englishh_df2)

pandas.core.frame.DataFrame

Now, experimenting with moving and renaming columns.  First mistake, forgot to set it as a new name.

In [64]:
englishhh_df2 = englishh_df2.rename(columns = {'ALink':'artist_name', 'SName':'song_name','SLink':'link','Lyric':'lyrics'})

In [71]:
englishhh_df2.head()

,artist_name,song_name,link,lyrics,Idiom
0,/10000-maniacs/,More Than This,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...,ENGLISH
1,/10000-maniacs/,Because The Night,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close...",ENGLISH
2,/10000-maniacs/,These Are Days,/10000-maniacs/these-are-days.html,These are. These are days you'll remember. Nev...,ENGLISH
3,/10000-maniacs/,A Campfire Song,/10000-maniacs/a-campfire-song.html,"A lie to say, ""O my mountain has coal veins an...",ENGLISH
4,/10000-maniacs/,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,Trudging slowly over wet sand. Back to the ben...,ENGLISH


Was looking to see the number of lyrics listed as instrumental.

In [72]:
englishhh_df2.lyrics.unique

<bound method Series.unique of 0         I could feel at the time. There was no way of ...
1         Take me now, baby, here as I am. Hold me close...
2         These are. These are days you'll remember. Nev...
3         A lie to say, "O my mountain has coal veins an...
4         Trudging slowly over wet sand. Back to the ben...
                                ...                        
207611    (Chorus). Hello, hello,hello,how low. Hello,he...
207624    Well sometimes I go out by myself. And I look ...
207628    Feeling my way through the darkness. Guided by...
207792    Don't, don't, that's what you say. Each time t...
209285    I've been. For almost everything in life. In t...
Name: lyrics, Length: 91619, dtype: object>

In [74]:
englishhh_df2.describe(include='all')

,artist_name,song_name,link,lyrics,Idiom
count,91619,91619,91619,91619,91619
unique,1198,70261,91608,90794,1
top,/elvis-presley/,Intro,/bee-gees/our-love-dont-throw-it-all-away.html,[This song is an instrumental.].,ENGLISH
freq,747,51,2,16,91619


In [80]:
englishhhh_df2 = englishhh_df2.DataFrame(data, columns='original.csv','artist_name','song_name','genre','date','link','lyrics')

SyntaxError: positional argument follows keyword argument (<ipython-input-80-f6d6d8f202bd>, line 1)

Move and create the columns IAW the final setup.

In [83]:
b

In [84]:
englishhhh_df2.head()

,original.csv,artist_name,song_name,genre,date,link,lyrics
0,NaN,/10000-maniacs/,More Than This,NaN,NaN,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...
1,NaN,/10000-maniacs/,Because The Night,NaN,NaN,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close..."
2,NaN,/10000-maniacs/,These Are Days,NaN,NaN,/10000-maniacs/these-are-days.html,These are. These are days you'll remember. Nev...
3,NaN,/10000-maniacs/,A Campfire Song,NaN,NaN,/10000-maniacs/a-campfire-song.html,"A lie to say, ""O my mountain has coal veins an..."
4,NaN,/10000-maniacs/,Everyday Is Like Sunday,NaN,NaN,/10000-maniacs/everyday-is-like-sunday.html,Trudging slowly over wet sand. Back to the ben...


In [85]:
englishhhh_df2['original.csv']='genres_lyrics-data.csv'

In [86]:
englishhhh_df2.head()

,original.csv,artist_name,song_name,genre,date,link,lyrics
0,genres_lyrics-data.csv,/10000-maniacs/,More Than This,NaN,NaN,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...
1,genres_lyrics-data.csv,/10000-maniacs/,Because The Night,NaN,NaN,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close..."
2,genres_lyrics-data.csv,/10000-maniacs/,These Are Days,NaN,NaN,/10000-maniacs/these-are-days.html,These are. These are days you'll remember. Nev...
3,genres_lyrics-data.csv,/10000-maniacs/,A Campfire Song,NaN,NaN,/10000-maniacs/a-campfire-song.html,"A lie to say, ""O my mountain has coal veins an..."
4,genres_lyrics-data.csv,/10000-maniacs/,Everyday Is Like Sunday,NaN,NaN,/10000-maniacs/everyday-is-like-sunday.html,Trudging slowly over wet sand. Back to the ben...


In [87]:
df1

,Artist,Songs,Popularity,Link,Genre,Genres
0,10000 Maniacs,110,0.3,/10000-maniacs/,Rock,Rock; Pop; Electronica; Dance; J-Pop/J-Rock; G...
1,12 Stones,75,0.3,/12-stones/,Rock,Rock; Gospel/Religioso; Hard Rock; Grunge; Roc...
2,311,196,0.5,/311/,Rock,Rock; Surf Music; Reggae; Ska; Pop/Rock; Rock ...
3,4 Non Blondes,15,7.5,/4-non-blondes/,Rock,Rock; Pop/Rock; Rock Alternativo; Grunge; Blue...
4,A Cruz Está Vazia,13,0.0,/a-cruz-esta-vazia/,Rock,Rock
...,...,...,...,...,...,...
3237,Péricles,102,6.8,/pericles/,Samba,Romântico; Pagode; Samba; Sertanejo; Samba Enr...
3238,Rodriguinho,106,2.7,/rodriguinho/,Samba,Romântico; Pagode; Samba; Country; Hardcore; T...
3239,Sambô,71,0.8,/sambo/,Rock,Samba; Pagode; Rock; Pop/Rock; Soul Music; Cla...
3240,Thiaguinho,143,13.8,/thiaguinho/,Samba,Pagode; Romântico; Samba; Trilha Sonora; Black...


In [153]:
df1.loc[0]['Link']

'/10000-maniacs/'

In [155]:
englishhhh_df2.loc[1]['artist_name']

'/10000-maniacs/'

In [88]:
englishhhh_df2.loc[4]['artist_name']

'/10000-maniacs/'

In [156]:
englishhhh_df2.loc[4]['artist_name']==df1.loc[0]['Link']

True

In [119]:
common_df = pd.DataFrame(df1, columns=['Link','Genre'])
common_df_no_dupes = common_df.drop_duplicates

In [121]:
common_df_no_dupes

<bound method DataFrame.drop_duplicates of                      Link  Genre
0         /10000-maniacs/   Rock
1             /12-stones/   Rock
2                   /311/   Rock
3         /4-non-blondes/   Rock
4     /a-cruz-esta-vazia/   Rock
...                   ...    ...
3237           /pericles/  Samba
3238        /rodriguinho/  Samba
3239              /sambo/   Rock
3240         /thiaguinho/  Samba
3241    /turma-do-pagode/  Samba

[3242 rows x 2 columns]>

In [122]:
common_df

,Link,Genre
0,/10000-maniacs/,Rock
1,/12-stones/,Rock
2,/311/,Rock
3,/4-non-blondes/,Rock
4,/a-cruz-esta-vazia/,Rock
...,...,...
3237,/pericles/,Samba
3238,/rodriguinho/,Samba
3239,/sambo/,Rock
3240,/thiaguinho/,Samba


In [116]:
common_df.Link.unique()

array(['/10000-maniacs/', '/12-stones/', '/311/', ..., '/rodriguinho/',
       '/thiaguinho/', '/turma-do-pagode/'], dtype=object)

In [117]:
common_df.describe(include='all')

,Link,Genre
count,3242,3242
unique,2940,6
top,/edu-gueda/,Rock
freq,3,797


In [123]:
common_df.Genre.unique()

array(['Rock', 'Pop', 'Hip Hop', 'Samba', 'Funk Carioca', 'Sertanejo'],
      dtype=object)

In [138]:
(df1.size)/(len(df1.keys()))

3242.0

In [139]:
x = 0
y = 0
e_var =(englishhhh_df2.size)/(len(englishhhh_df2.keys()))
ee_var = int(e_var)
d_var = (df1.size)/(len(df1.keys()))
dd_var = int(d_var)

for i in range (0, ee_var):
	artist = englishhhh_df2.loc[x]['artist_name']
	for q in range (0, dd_var):
		if artist == df1.loc[y]['Link'] is True:
			englishhhh_df2.loc[x]['genre'] = df1.loc[y]['Genre']
			break
		else:
			y = y + 1
	x = x +1

return englishhhh_df2

TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [143]:
e_var =(englishhhh_df2.size)/(len(englishhhh_df2.keys()))
ee_var = int(e_var)

91619.0

In [144]:
ee_var = int(e_var)
type(ee_var)

int

In [145]:
d_var = (df1.size)/(len(df1.keys()))
dd_var = int(d_var)
type(dd_var)

int

In [146]:
d_var

3242.0

In [147]:
dd_var

3242

In [148]:
e_var =int((englishhhh_df2.size)/(len(englishhhh_df2.keys())))
type(e_var)

int

In [150]:
x = 0
y = 0
e_var =int((englishhhh_df2.size)/(len(englishhhh_df2.keys())))
d_var = int((df1.size)/(len(df1.keys())))

for i in range (0, e_var):
	artist = englishhhh_df2.loc[x]['artist_name']
	for q in range (0, d_var):
		if artist == df1.loc[y]['Link'] is True:
			englishhhh_df2.loc[x]['genre'] = df1.loc[y]['Genre']
			break
		else:
			y = y + 1
	x = x +1

return englishhhh_df2

KeyError: 3242

In [151]:
d_var = int((df1.size)/(len(df1.keys())))
d_var

3242

In [152]:
type(d_var)

int

In [161]:
x = 0
y = 0
e_var =int((englishhhh_df2.size)/(len(englishhhh_df2.keys())))
d_var = int((df1.size)/(len(df1.keys())))

for i in range (0, e_var):
	artist = englishhhh_df2.loc[x]['artist_name']
	for q in range (0, d_var):
		if artist == df1.loc[y]['Link']:
			englishhhh_df2.loc[x]['genre'] = df1.loc[y]['Genre']
			break
		else:
			y = y + 1
	x = x +1

return englishhhh_df2

<ipython-input-161-2536af0bb033>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  englishhhh_df2.loc[x]['genre'] = df1.loc[y]['Genre']


KeyError: 23

So, I have no idea what that error means, really. So, that's hard to fix.  And I've fixed a lot of things trying to get to this point.

In [163]:
if englishhhh_df2.loc[4]['artist_name']==df1.loc[0]['Link']:
    print('hit')
    englishhhh_df2.loc[4]['genre']=df1.loc[0]['Genre']
    print(df1.loc[0]['Genre'])
    print(englishhhh_df2.loc[4]['genre']) 
else:
    print('miss')

hit
Rock
nan


<ipython-input-163-ee2df10b60b4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  englishhhh_df2.loc[4]['genre']=df1.loc[0]['Genre']


In [164]:
common_df

,Link,Genre
0,/10000-maniacs/,Rock
1,/12-stones/,Rock
2,/311/,Rock
3,/4-non-blondes/,Rock
4,/a-cruz-esta-vazia/,Rock
...,...,...
3237,/pericles/,Samba
3238,/rodriguinho/,Samba
3239,/sambo/,Rock
3240,/thiaguinho/,Samba


In [165]:
englishhhh_df2

,original.csv,artist_name,song_name,genre,date,link,lyrics
0,genres_lyrics-data.csv,/10000-maniacs/,More Than This,NaN,NaN,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...
1,genres_lyrics-data.csv,/10000-maniacs/,Because The Night,NaN,NaN,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close..."
2,genres_lyrics-data.csv,/10000-maniacs/,These Are Days,NaN,NaN,/10000-maniacs/these-are-days.html,These are. These are days you'll remember. Nev...
3,genres_lyrics-data.csv,/10000-maniacs/,A Campfire Song,NaN,NaN,/10000-maniacs/a-campfire-song.html,"A lie to say, ""O my mountain has coal veins an..."
4,genres_lyrics-data.csv,/10000-maniacs/,Everyday Is Like Sunday,NaN,NaN,/10000-maniacs/everyday-is-like-sunday.html,Trudging slowly over wet sand. Back to the ben...
...,...,...,...,...,...,...,...
207611,genres_lyrics-data.csv,/sambo/,Smells Like Teen Spirit,NaN,NaN,/sambo/smells-like-teen-spirit.html,"(Chorus). Hello, hello,hello,how low. Hello,he..."
207624,genres_lyrics-data.csv,/sambo/,Valerie,NaN,NaN,/sambo/valerie.html,Well sometimes I go out by myself. And I look ...
207628,genres_lyrics-data.csv,/sambo/,Wake Me Up,NaN,NaN,/sambo/wake-me-up.html,Feeling my way through the darkness. Guided by...
207792,genres_lyrics-data.csv,/seu-jorge/,Don't,NaN,NaN,/seu-jorge/dont.html,"Don't, don't, that's what you say. Each time t..."


In [166]:
new_common_df = common_df.rename(columns={'Link':'artist_name','Genre':'genre'})
new_common_df

,artist_name,genre
0,/10000-maniacs/,Rock
1,/12-stones/,Rock
2,/311/,Rock
3,/4-non-blondes/,Rock
4,/a-cruz-esta-vazia/,Rock
...,...,...
3237,/pericles/,Samba
3238,/rodriguinho/,Samba
3239,/sambo/,Rock
3240,/thiaguinho/,Samba


In [167]:
englishhhhh_df2 = englishhhh_df2.drop(['genre'], axis = 1)

In [168]:
englishhhhh_df2

,original.csv,artist_name,song_name,date,link,lyrics
0,genres_lyrics-data.csv,/10000-maniacs/,More Than This,NaN,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...
1,genres_lyrics-data.csv,/10000-maniacs/,Because The Night,NaN,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close..."
2,genres_lyrics-data.csv,/10000-maniacs/,These Are Days,NaN,/10000-maniacs/these-are-days.html,These are. These are days you'll remember. Nev...
3,genres_lyrics-data.csv,/10000-maniacs/,A Campfire Song,NaN,/10000-maniacs/a-campfire-song.html,"A lie to say, ""O my mountain has coal veins an..."
4,genres_lyrics-data.csv,/10000-maniacs/,Everyday Is Like Sunday,NaN,/10000-maniacs/everyday-is-like-sunday.html,Trudging slowly over wet sand. Back to the ben...
...,...,...,...,...,...,...
207611,genres_lyrics-data.csv,/sambo/,Smells Like Teen Spirit,NaN,/sambo/smells-like-teen-spirit.html,"(Chorus). Hello, hello,hello,how low. Hello,he..."
207624,genres_lyrics-data.csv,/sambo/,Valerie,NaN,/sambo/valerie.html,Well sometimes I go out by myself. And I look ...
207628,genres_lyrics-data.csv,/sambo/,Wake Me Up,NaN,/sambo/wake-me-up.html,Feeling my way through the darkness. Guided by...
207792,genres_lyrics-data.csv,/seu-jorge/,Don't,NaN,/seu-jorge/dont.html,"Don't, don't, that's what you say. Each time t..."


In [176]:
df4_right = pd.merge(englishhhhh_df2,new_common_df,on='artist_name',how='right')

In [177]:
df4_right

,original.csv,artist_name,song_name,date,link,lyrics,genre
0,genres_lyrics-data.csv,/10000-maniacs/,More Than This,NaN,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...,Rock
1,genres_lyrics-data.csv,/10000-maniacs/,Because The Night,NaN,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close...",Rock
2,genres_lyrics-data.csv,/10000-maniacs/,These Are Days,NaN,/10000-maniacs/these-are-days.html,These are. These are days you'll remember. Nev...,Rock
3,genres_lyrics-data.csv,/10000-maniacs/,A Campfire Song,NaN,/10000-maniacs/a-campfire-song.html,"A lie to say, ""O my mountain has coal veins an...",Rock
4,genres_lyrics-data.csv,/10000-maniacs/,Everyday Is Like Sunday,NaN,/10000-maniacs/everyday-is-like-sunday.html,Trudging slowly over wet sand. Back to the ben...,Rock
...,...,...,...,...,...,...,...
96885,genres_lyrics-data.csv,/sambo/,Smells Like Teen Spirit,NaN,/sambo/smells-like-teen-spirit.html,"(Chorus). Hello, hello,hello,how low. Hello,he...",Rock
96886,genres_lyrics-data.csv,/sambo/,Valerie,NaN,/sambo/valerie.html,Well sometimes I go out by myself. And I look ...,Rock
96887,genres_lyrics-data.csv,/sambo/,Wake Me Up,NaN,/sambo/wake-me-up.html,Feeling my way through the darkness. Guided by...,Rock
96888,NaN,/thiaguinho/,NaN,NaN,NaN,NaN,Samba


In [172]:
new_common_df[new_common_df.duplicated()]

,artist_name,genre


In [175]:
df4.head(100)

,original.csv,artist_name,song_name,date,link,lyrics,genre
0,genres_lyrics-data.csv,/10000-maniacs/,More Than This,NaN,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...,Rock
1,genres_lyrics-data.csv,/10000-maniacs/,Because The Night,NaN,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close...",Rock
2,genres_lyrics-data.csv,/10000-maniacs/,These Are Days,NaN,/10000-maniacs/these-are-days.html,These are. These are days you'll remember. Nev...,Rock
3,genres_lyrics-data.csv,/10000-maniacs/,A Campfire Song,NaN,/10000-maniacs/a-campfire-song.html,"A lie to say, ""O my mountain has coal veins an...",Rock
4,genres_lyrics-data.csv,/10000-maniacs/,Everyday Is Like Sunday,NaN,/10000-maniacs/everyday-is-like-sunday.html,Trudging slowly over wet sand. Back to the ben...,Rock
...,...,...,...,...,...,...,...
95,genres_lyrics-data.csv,/10000-maniacs/,The Latin One,NaN,/10000-maniacs/the-latin-one.html,Bent double like old beggars in sacks. Knockkn...,Rock
96,genres_lyrics-data.csv,/10000-maniacs/,The Lion's Share,NaN,/10000-maniacs/the-lions-share.html,[ music: Natalie Merchant & Dennis Drew/words:...,Rock
97,genres_lyrics-data.csv,/10000-maniacs/,The Painted Desert,NaN,/10000-maniacs/the-painted-desert.html,The Painted Desert can wait till summer.. We'v...,Rock
98,genres_lyrics-data.csv,/10000-maniacs/,These days,NaN,/10000-maniacs/these-days.html,[ Jackson Browne ]. . I've been out walking. I...,Rock


In [178]:
df4_left = pd.merge(englishhhhh_df2,new_common_df,on='artist_name',how='left')

In [180]:
df4_left

,original.csv,artist_name,song_name,date,link,lyrics,genre
0,genres_lyrics-data.csv,/10000-maniacs/,More Than This,NaN,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...,Rock
1,genres_lyrics-data.csv,/10000-maniacs/,More Than This,NaN,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...,Pop
2,genres_lyrics-data.csv,/10000-maniacs/,Because The Night,NaN,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close...",Rock
3,genres_lyrics-data.csv,/10000-maniacs/,Because The Night,NaN,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close...",Pop
4,genres_lyrics-data.csv,/10000-maniacs/,These Are Days,NaN,/10000-maniacs/these-are-days.html,These are. These are days you'll remember. Nev...,Rock
...,...,...,...,...,...,...,...
99367,genres_lyrics-data.csv,/sambo/,Valerie,NaN,/sambo/valerie.html,Well sometimes I go out by myself. And I look ...,Rock
99368,genres_lyrics-data.csv,/sambo/,Wake Me Up,NaN,/sambo/wake-me-up.html,Feeling my way through the darkness. Guided by...,Samba
99369,genres_lyrics-data.csv,/sambo/,Wake Me Up,NaN,/sambo/wake-me-up.html,Feeling my way through the darkness. Guided by...,Rock
99370,genres_lyrics-data.csv,/seu-jorge/,Don't,NaN,/seu-jorge/dont.html,"Don't, don't, that's what you say. Each time t...",Samba


In [181]:
df4_blank = pd.merge(englishhhhh_df2,new_common_df,on='artist_name')

In [182]:
df4_blank

,original.csv,artist_name,song_name,date,link,lyrics,genre
0,genres_lyrics-data.csv,/10000-maniacs/,More Than This,NaN,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...,Rock
1,genres_lyrics-data.csv,/10000-maniacs/,More Than This,NaN,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...,Pop
2,genres_lyrics-data.csv,/10000-maniacs/,Because The Night,NaN,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close...",Rock
3,genres_lyrics-data.csv,/10000-maniacs/,Because The Night,NaN,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close...",Pop
4,genres_lyrics-data.csv,/10000-maniacs/,These Are Days,NaN,/10000-maniacs/these-are-days.html,These are. These are days you'll remember. Nev...,Rock
...,...,...,...,...,...,...,...
94907,genres_lyrics-data.csv,/sambo/,Valerie,NaN,/sambo/valerie.html,Well sometimes I go out by myself. And I look ...,Rock
94908,genres_lyrics-data.csv,/sambo/,Wake Me Up,NaN,/sambo/wake-me-up.html,Feeling my way through the darkness. Guided by...,Samba
94909,genres_lyrics-data.csv,/sambo/,Wake Me Up,NaN,/sambo/wake-me-up.html,Feeling my way through the darkness. Guided by...,Rock
94910,genres_lyrics-data.csv,/seu-jorge/,Don't,NaN,/seu-jorge/dont.html,"Don't, don't, that's what you say. Each time t...",Samba


In [183]:
new_common_df.describe(include='all')

,artist_name,genre
count,3242,3242
unique,2940,6
top,/edu-gueda/,Rock
freq,3,797
